In [18]:
import random
import itertools

import pandas

In [19]:
DATA_PATH: str = "../../../data"
DATASET_PATH: str = f"{DATA_PATH}/processed/twitter.german.dataset.enriched.csv"

In [20]:
data = pandas.read_csv(DATASET_PATH, index_col=[0])
data.head()

,id_post,id_reply,author_id_post,author_id_reply,author_first_name_post,author_last_name_post,author_party_post,text_post,text_reply,topics_post,topics_reply
0,1616701560431542272,1617114281132314624,42698498,15891726,Roderich,Kiesewetter,CDU/CSU,"Aus meinem Wahlkreis heute eine Bürgerfrage: ""...","Wussten das die CDU Verteidigungsminister, die...","Wahlkreis,Bürgerfrage,Gewehre","Wehretat, CDU, Verteidigungsminister"
1,1620171503957581825,1620308520087199744,713361366858481664,15891726,Stefan,Brandner,AfD,Zeigt der #staatsfunk etwa gerade einen #coron...,Da es keinen Staatsfunk gibt: Nein.,"Film, Corona, Staatsfernsehen","Deutsche Staatsbürgerschaft, öffentliche Rundf..."
2,1649820541207863300,1649890068067151878,18189342,15891726,Frank,Schäffler,FDP,FDP will den Weiterbetrieb der Kernenergie in ...,"Steht da auch drinne, dass die Dinger ohne Sub...","Kernenergie, FDP-Politik, AKW-Rückbau","Bildung, Politik, Finanzen"
3,1610260317262471171,1610903010426908673,712941191,15891726,Katja,Adler,FDP,Wir müssen eine Debatte zur Migrationspolitik ...,Leute die Begriffe wie „kulturelle Überfremdun...,"Migrationspolitik, politische Sprache, öffentl...","Kulturelle Überfremdung, politische Sprache, D..."
4,1657692554706534401,1657718828485206016,713361366858481664,15891726,Stefan,Brandner,AfD,"Jedenfalls haben ""Lord of the Lost""🤪 (der Name...",Oh doch. Zum Beispiel könnte eine AfD Band Nam...,"Esc, Musik, Lord of the Lost","Politik, Musik, Satire"


In [21]:
positive_samples = {
    author: [
        sample for sample in itertools.combinations(group["text_post"].tolist(), 3)
    ]
    for author, group in data.groupby("author_id_reply")
}
len(positive_samples)

348

In [22]:
negative_samples = {
    author: [
        (
            *random.choice(positive_samples[author])[:2],
            random.choice(
                positive_samples[
                    random.choice(list(set(positive_samples.keys() - set([author]))))
                ]
            )[2],
        )
        for _ in range(450)
    ]
    for author in set(positive_samples.keys())
}
len(negative_samples)

348

In [29]:
dataset = pandas.concat(
    [
        pandas.DataFrame(
            [
                dict(
                    history_1=sample[0],
                    history_2=sample[1],
                    stimulus=sample[2],
                    action=label,
                )
                for group in sets
                for sample in group
            ]
        ).sample(10_000)
        for label, sets in [
            (True, positive_samples.values()),
            (False, negative_samples.values()),
        ]
    ]
)

dataset.to_csv(DATASET_PATH.replace("enriched", "decision"))